In [7]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [9]:
# Extract data from the PDFs

def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls = PyPDFLoader)
    
    documents = loader.load()

    return documents

In [12]:
extracted_data = load_pdf("../data/")

In [13]:
# Creating data chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                   chunk_overlap = 20)
    
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)

In [15]:
print(f"Length of chunks is {len(text_chunks)}")

Length of chunks is 7020


In [16]:
# Download Embedding Models

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name ="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
    

In [17]:
embeddings = download_hugging_face_embedding()

In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [19]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [20]:
print(query_result)

[-0.03447730466723442, 0.03102327696979046, 0.0067350068129599094, 0.026109062135219574, -0.03936205431818962, -0.16030248999595642, 0.06692402809858322, -0.006441421806812286, -0.047450464218854904, 0.014758830890059471, 0.07087530940771103, 0.055527493357658386, 0.019193366169929504, -0.026251330971717834, -0.010109709575772285, -0.026940545067191124, 0.022307490929961205, -0.022226646542549133, -0.1496925801038742, -0.017493097111582756, 0.007676301058381796, 0.054352376610040665, 0.0032545067369937897, 0.03172590956091881, -0.08462141454219818, -0.02940606325864792, 0.051595669239759445, 0.04812396690249443, -0.0033148089423775673, -0.05827924981713295, 0.04196939989924431, 0.022210612893104553, 0.128188818693161, -0.02233898639678955, -0.011656300164759159, 0.06292829662561417, -0.03287627920508385, -0.09122608602046967, -0.031175274401903152, 0.052699558436870575, 0.04703487083315849, -0.08420301228761673, -0.030056163668632507, -0.020744873210787773, 0.00951786246150732, -0.0037

In [22]:
pinecone.init(api_key= PINECONE_API_KEY,
              environment= PINECONE_API_ENV)

index_name  = "medical-chatbot"

docsearch = Pinecone.from_texts([text.page_content for text in text_chunks], embeddings, index_name=index_name)

In [24]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k =3)

print("Results: ", docs)

Results:  [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\

In [25]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
"""

In [26]:
PROMPT = PromptTemplate(template= prompt_template, input_variables= ["context","question"])
chain_type_kwargs = {'prompt' : PROMPT}

In [28]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [29]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [31]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
